In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [2]:
print(pyspark.__version__)
 


3.2.3


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.mongodb.input.uri", "mongodb://Ubuntu:27017/trumpDB")
spark.conf.set("spark.mongodb.output.uri", "mongodb://Ubuntu:27017/trumpDB")

df = spark.read.format("mongo").load()


IllegalArgumentException: Missing database name. Set via the 'spark.mongodb.input.uri' or 'spark.mongodb.input.database' property

In [7]:
df = spark.read.format("mongo").option("uri", "mongodb://Ubuntu:27017/trumpDB.trumpCollection").load()
df = spark.read.format("mongo").option("uri", "mongodb://Ubuntu:27017/trumpDB.trumpCollection").load()


Py4JJavaError: An error occurred while calling o57.load.
: com.mongodb.MongoIncompatibleDriverException: Server at ubuntu:27017 reports wire version 4, but this version of the driver requires at least 6 (MongoDB 3.6).
	at com.mongodb.internal.connection.BaseCluster.createIncompatibleException(BaseCluster.java:424)
	at com.mongodb.internal.connection.BaseCluster.throwIfIncompatible(BaseCluster.java:404)
	at com.mongodb.internal.connection.BaseCluster.selectServer(BaseCluster.java:118)
	at com.mongodb.internal.connection.SingleServerCluster.selectServer(SingleServerCluster.java:46)
	at com.mongodb.internal.binding.ClusterBinding.getReadConnectionSource(ClusterBinding.java:108)
	at com.mongodb.internal.operation.OperationHelper.withSuppliedResource(OperationHelper.java:373)
	at com.mongodb.internal.operation.OperationHelper.withSourceAndConnection(OperationHelper.java:354)
	at com.mongodb.internal.operation.CommandOperationHelper.lambda$executeRetryableRead$5(CommandOperationHelper.java:2

In [4]:
# Set MongoDB connection configuration
spark.conf.set("spark.mongodb.input.uri", "mongodb://Ubuntu:27017/trumpDB.")
spark.conf.set("spark.mongodb.output.uri", "mongodb://Ubuntu:27017/trumpDB.")

# Load data from MongoDB into DataFrame
df = spark.read.format("mongo").load()

# Show the DataFrame
df.show()


IllegalArgumentException: Missing database name. Set via the 'spark.mongodb.input.uri' or 'spark.mongodb.input.database' property

In [ ]:
filename = '2016_US_election_tweets.csv'

# Read input file from hadoop directory and convert to pandas
df = spark.read.csv('/user1/CA2/'+filename,header=True)


In [ ]:
print(df.columns)

In [ ]:
df = df.select('candidate_id', "created_at", "tweet_text")

In [ ]:
df_no_nulls = df.na.drop()
tweets = df_no_nulls.filter(df.candidate_id == 2)

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    return analyzer.polarity_scores(text)["compound"]

sentiment_udf = udf(analyze_sentiment, FloatType())
spark.udf.register("sentiment_udf", sentiment_udf)

In [ ]:
# Convert timestamp to date
tweets = tweets.withColumn("date", to_date(col("created_at")))

# Calculate sentiment score for each tweet
tweets = tweets.withColumn("sentiment", sentiment_udf(col("tweet_text")))


In [ ]:
tweets = tweets.select('tweet_text','date','sentiment')

tweets.show()

In [ ]:
from pyspark.sql.functions import year

# filter out out-of-range dates
tweets = tweets.filter((year("date") >= 1) & (year("date") <= 9999))

In [ ]:
daily_sentiment_df = tweets.groupBy("date") \
    .agg(F.mean("sentiment").alias("avg_sentiment")) \
    .orderBy("date")

In [ ]:
daily_sentiment_pd = daily_sentiment_df.toPandas()

In [ ]:
df2 = daily_sentiment_pd.iloc[:194,:]

In [ ]:
df2['date'] = pd.to_datetime(df2['date'])

In [ ]:
print(df2['date'])

In [ ]:
df2 = df2[(df2['date'].dt.year >= 2016) & (df2['date'].dt.year <= 2017)]

In [ ]:
print(df2)

In [ ]:
df2 = df2.iloc[1:]

In [ ]:
plt.plot(df2['date'],df2['avg_sentiment'])

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(x = df2.date,y = df2.avg_sentiment)
fig.show()


In [ ]:
[print(x) for x in df2['date']] 